In [1]:
from lib.util import *
from lib.policy import *
from lib.mdp import *
from lib.mrp import *

# Environment

In [2]:
from lib.env import *

# Tabular Monte Carlo

In [3]:
from lib.monte_carlo import *

$$V(S_t) \leftarrow V(S_t) + \alpha (G_t - V(S_t))$$

# Tabular Temporal Difference

In [4]:
from lib.temporal_difference import *

- Update value V(St) toward estimated return $R_{t+1} + γV(S_{t+1})$:
$$V(S_t) ← V(S_t) + α (R_{t+1} + γV(S_{t+1}) − V(S_t))$$
- $R_{t+1} + γV(S_{t+1})$ is called the TD target
- $δ_t = R_{t+1} + γV(S_{t+1}) − V(S_t)$ is called the TD error

# Example

###### MDP

In [5]:
n = 5
gamma = 0.95

In [6]:
P = generate_stochastic_matrix(n)
R = generate_reward_vector(n)
mrp = MRP(P, R, gamma)
mdp = MDP(gamma, [mrp]*n)
Q = generate_stochastic_matrix(n)
policy = Policy(Q)

###### Environment

In [7]:
env = Env(mdp)
print(env.reset())
print(env.step(0, 1))
print(env.step(2, 1))

1
(1, 0.9498277274225383, False, None)
(0, 0.46812031720758607, False, None)


###### Algorithm

In [8]:
env = Env(mdp)

In [9]:
monte_carlo(env, policy)

defaultdict(float,
            {3: 20.01469063997351,
             0: 20.546815743326817,
             1: 19.85233826599915,
             2: 20.54701960644944,
             4: 19.813251662907135})

In [10]:
td_zero(env, policy)

defaultdict(float,
            {1: 6.364887418515749,
             4: 6.336469069480168,
             2: 6.9377047380869765,
             3: 6.8566856480784715,
             0: 7.375639492906918})

In [11]:
forward_td_lambda(env, policy)

defaultdict(float,
            {3: 7.242193425893754,
             1: 7.023605886027626,
             4: 6.937574505044491,
             0: 6.608214027859459,
             2: 6.413893756622728})

In [12]:
backward_td_lambda(env, policy)

defaultdict(float,
            {1: 7.5198029667541,
             3: 7.513546264050374,
             4: 6.743670784989685,
             2: 6.859958600577809,
             0: 7.246732626133098})

### Proof that fixed learning rate $\alpha$ for MC is equivalent to an exponentially decaying average of episode returns

\begin{align*}
\mu_k &= \mu_{k-1} + \alpha(x_k - \mu_{k-1}) \\
&= \alpha x_k + (1-\alpha)\mu_{k-1} \\
&= \alpha x_k + (1-\alpha)(\alpha x_{k-1} + (1-\alpha)\mu_{k-2}) \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2\mu_{k-2} \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2(\alpha x_{k-2} + (1-\alpha)\mu_{k-3}) \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2\alpha x_{k-2} + (1-\alpha)^3\mu_{k-3} \\
&= \dots \\
&= (1-\alpha)^kx_0 + \alpha\sum_{i=1}^k (1-\alpha)^{k-i} x_i
\end{align*}

### Proof that Offline Forward-View TD($\lambda$) and Offline Backward View TD($\lambda$) are equivalent

$E_t(s) = \gamma \lambda E_{t-1}(s) + \mathbb{1}(S_t = s) = (\gamma \lambda)^{t-k}\mathbb{1}(t \geq k)$

Backward $TD(\lambda)$ updates accumulate error online $\sum_{t=1}^T \alpha \delta_t E_t(s) = \alpha\sum_{t=k}^T(\gamma \lambda)^{t-k}\delta_t $

Moreover,
\begin{align*}
G_t^\lambda - V(S_t) 
&= -V(S_t) + (1-\lambda)\lambda^0 (R_{t+1}+\gamma V(S_{t+1})) + (1-\lambda)\lambda^1 (R_{t+1}+\gamma R_{t+2} +\gamma^2 V(S_{t+2})) + \dots \\
&= -V(S_t) + (\gamma \lambda)^0 (R_{t+1}+\gamma V(S_{t+1}) - \gamma\lambda V(S_{t+1})) + (\gamma \lambda)^1 (R_{t+2}+\gamma V(S_{t+2}) - \gamma\lambda V(S_{t+2})) + \dots \\
&= (\gamma \lambda)^0 (R_{t+1}+\gamma V(S_{t+1}) - V(S_{t})) + (\gamma \lambda)^1 (R_{t+2}+\gamma V(S_{t+3}) - V(S_{t+2})) + \dots \\
&= \sum_{t=k}^T(\gamma \lambda)^{t-k}\delta_t
\end{align*}

Hence, we get  $\sum_{t=1}^T \alpha \delta_t E_t(s) =  \sum_{t=1}^T \alpha (G_t^\lambda - V(S_t))$